# 挂载google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmplwy9lwvk/pubring.gpg' created
gpg: /tmp/tmplwy9lwvk/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
!pwd
import os
os.chdir('./drive/COMP5318/comp5318 assignment2/')
!pwd
!ls

/content
/content/drive/COMP5318/comp5318 assignment2
comp5318_assignment2_PCA_RF.ipynb（副本）  dataset  drive  共享


#assignment

In [0]:
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
import scipy.io as scio
from sklearn.decomposition import PCA
import numpy as np
import time
import gc

##PCA

In [0]:
#PCA part
train_path = "./dataset/train_32x32.mat"
data = scio.loadmat(train_path)

train_X = data["X"]
train_y = data["y"]
train_X = train_X.T
print(train_X.shape)
print(train_y.shape)

tmp = []
for row in train_X:
    tmp.append(row.flatten())
# release training set memory
train_X = None
gc.collect()

train_X = np.array(tmp)
tmp = None

train_X = train_X - train_X.mean(axis=0)
train_X.shape

# PCA decrease the dimension of the training set into d dimensions.
# d is a configurable variable (hyperparameter)
# In this case d = 50.
d = 50
pca = PCA(n_components=d)
start = time.time()
train_X = pca.fit(train_X).transform(train_X)
end = time.time()
print("shape: {}".format(str(train_X.shape)))
print("time consumption: {}s".format(end-start))
dataset = np.hstack((train_X,train_y))

(73257, 3, 32, 32)
(73257, 1)
shape: (73257, 50)
time consumption: 33.56474757194519s


##定义K折

###onehot sklearn不要

In [0]:
from sklearn.preprocessing import OneHotEncoder
# Fit the OneHotEncoder
enc = OneHotEncoder().fit(train_y.reshape(-1, 1))
# Transform the label values to a one-hot-encoding scheme
train_y_one_hot = enc.transform(train_y.reshape(-1, 1)).toarray()


In [0]:


def train_test_split_10(train_X, train_y,k):
    num_of_examples = len(train_X)
    start_jump = (num_of_examples//10)*k
    end_jump = (num_of_examples//10)*(k+1)
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]
    for i in range(len(train_X)):
        if i<=start_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        elif i>end_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        else:
            X_test.append(train_X[i])
            y_test.append(train_y[i])
    print(np.shape(X_train))
    print(np.shape(y_train))
    print(np.shape(X_test))
    print(np.shape(y_test))

    return X_train,X_test,y_train,y_test

X_train, X_test, y_train, y_test =  train_test_split_10(train_X, train_y,0)




(65932, 50)
(65932, 1)
(7325, 50)
(7325, 1)


##RF tensorflow

In [0]:
def train(X_train,y_train , X_test, y_test):
    # RF
    tf.reset_default_graph()
    num_etsp = 1000
    batch_size = 1000
    num_class = 10
    num_feature = 50
    num_tree = 100
    max_nodes = 1000

    # parameter part
    X = tf.placeholder(tf.float32, shape=[None, d])
    Y = tf.placeholder(tf.float32, shape=[None, num_class])

    hparams = tensor_forest.ForestHParams(num_features=num_feature, num_classes=num_class, num_trees=num_tree,
                                          max_nodes=max_nodes).fill()

    forest_Graph = tensor_forest.RandomForestGraphs(hparams)
    train_op = forest_Graph.training_graph(X, Y)
    loss_op = forest_Graph.training_loss(X, Y)

    infer_op, _, _ = forest_Graph.inference_graph(X)
    correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.argmax(Y, 1))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init_val = tf.group(tf.global_variables_initializer(), resources.initialize_resources(resources.shared_resources()))

    with tf.Session() as sess:
        sess.run(init_val)
        batch_current = 0
        batch_end = len(X_train)
        for i in range(1, num_etsp + 1):
            if batch_current + batch_size > batch_end:
                xs = X_train[batch_end - batch_size:batch_end]
                ys = y_train[batch_end - batch_size:batch_end]
            else:
                xs = X_train[batch_current:min(batch_current + batch_size, batch_end)]
                ys = y_train[batch_current:min(batch_current + batch_size, batch_end)]
            _, l = sess.run([train_op, loss_op], feed_dict={X: xs, Y: ys})
            batch_current = min(batch_current + batch_size, batch_end)
            if i % 50 == 0 or i == 1:
                acc = sess.run(accuracy_op, feed_dict={X: xs, Y: ys})

        print(sess.run(accuracy_op, feed_dict={X: X_test, Y: y_test}))


In [0]:
train(X_train,y_train,X_test,y_test)

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 100, 'max_nodes': 1000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_features': 50, 'num_classes': 10, 'bagged_num_features': 50, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 11, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}
0.1883959


###测试

In [0]:
start=time.time()
for i in range(10):
  X_train, X_test, y_train, y_test =  train_test_split_10(train_X, train_y,i)
  train(X_train,y_train,X_test,y_test)
end =time.time()
print("time cost = ".format(end-start))

(65932, 50)
(65932, 1)
(7325, 50)
(7325, 1)


NameError: ignored

##RF sklearn

默认参数

In [0]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test =  train_test_split_10(train_X, train_y,0)
y_train_num=[]
for i in range(len(y_train)):
  label = y_train[i][0]
  #print(label)
  y_train_num.append(label)
y_train=np.array(y_train_num)

y_test_num=[]
for i in range(len(y_test)):
  label = y_test[i][0]
  #print(label)
  y_test_num.append(label)
y_test=np.array(y_test_num)

print(y_train.shape)
print(y_test.shape)

rf0 = RandomForestClassifier(oob_score=True, random_state=10)  
rf0.fit(X_train,y_train)  
print (rf0.oob_score_)  
y_predprob = rf0.predict_proba(X_test)

#print(np.shape(y_test))
#print(np.shape(y_predprob))

score = cross_val_score(rf0, train_X, train_y, cv=10, scoring='accuracy').mean()
print('Validation score is', score)

(65932, 50)
(65932, 1)
(7325, 50)
(7325, 1)
(65932,)
(7325,)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


0.34664502821088394


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/skle

Validation score is 0.4533216974125643


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


对n_estimators进行网格搜索

In [0]:
from sklearn.model_selection import GridSearchCV

param_test1= {'n_estimators': range(10,71,10)}

gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,  
                                 min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10),
                        param_grid=param_test1,scoring='accuracy',cv=5) 
gsearch1.fit(X_train,y_train)  
gsearch1.grid_scores_,gsearch1.best_params_, gsearch1.best_score_  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.36991, std: 0.00287, params: {'n_estimators': 10},
  mean: 0.37587, std: 0.00052, params: {'n_estimators': 20},
  mean: 0.37942, std: 0.00230, params: {'n_estimators': 30},
  mean: 0.38013, std: 0.00242, params: {'n_estimators': 40},
  mean: 0.38039, std: 0.00294, params: {'n_estimators': 50},
  mean: 0.37977, std: 0.00261, params: {'n_estimators': 60},
  mean: 0.38004, std: 0.00207, params: {'n_estimators': 70}],
 {'n_estimators': 50},
 0.38039191894679364)

得到了最佳的弱学习器迭代次数，接着我们对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索

In [0]:
start = time.time()
param_test2= {'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)}  
gsearch2= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 50,  
                                 min_samples_leaf=20,max_features='sqrt' ,oob_score=True,random_state=10),  
   param_grid = param_test2,scoring='accuracy',iid=False, cv=5)  
gsearch2.fit(X_train,y_train)  
gsearch2.grid_scores_,gsearch2.best_params_, gsearch2.best_score_  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 50},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 70},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 90},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 110},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 130},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 150},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 170},
  mean: 0.23245, std: 0.00245, params: {'max_depth': 3, 'min_samples_split': 190},
  mean: 0.29547, std: 0.00228, params: {'max_depth': 5, 'min_samples_split': 50},
  mean: 0.29439, std: 0.00210, params: {'max_depth': 5, 'min_samples_split': 70},
  mean: 0.29468, std: 0.00225, params: {'max_depth': 5, 'min_samples_split': 90},
  mean: 0.29435, std: 0.00318, params: {'max_depth': 5, 'min_samples_split': 110},
  mean: 0.

取了三个最优参数，看看现在模型的袋外分数

In [0]:
rf1= RandomForestClassifier(n_estimators= 50, max_depth=13, min_samples_split=50,  
                                 min_samples_leaf=20,max_features='sqrt' ,oob_score=True,random_state=10)  
rf1.fit(X_train,y_train)  
print(rf1.oob_score_)  

0.4819055997087909


对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参

In [0]:
param_test3= {'min_samples_split':range(80,150,20), 'min_samples_leaf':range(10,60,10)}  
gsearch3= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 50,max_depth=13,  
                                 max_features='sqrt' ,oob_score=True, random_state=10),  
   param_grid = param_test3,scoring='accuracy',iid=False, cv=5)  
gsearch3.fit(X_train,y_train)  
gsearch3.grid_scores_,gsearch2.best_params_, gsearch2.best_score_  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.47786, std: 0.00292, params: {'min_samples_leaf': 10, 'min_samples_split': 80},
  mean: 0.47132, std: 0.00225, params: {'min_samples_leaf': 10, 'min_samples_split': 100},
  mean: 0.46020, std: 0.00430, params: {'min_samples_leaf': 10, 'min_samples_split': 120},
  mean: 0.45304, std: 0.00327, params: {'min_samples_leaf': 10, 'min_samples_split': 140},
  mean: 0.47482, std: 0.00311, params: {'min_samples_leaf': 20, 'min_samples_split': 80},
  mean: 0.46572, std: 0.00410, params: {'min_samples_leaf': 20, 'min_samples_split': 100},
  mean: 0.45937, std: 0.00312, params: {'min_samples_leaf': 20, 'min_samples_split': 120},
  mean: 0.45118, std: 0.00103, params: {'min_samples_leaf': 20, 'min_samples_split': 140},
  mean: 0.47450, std: 0.00098, params: {'min_samples_leaf': 30, 'min_samples_split': 80},
  mean: 0.46490, std: 0.00127, params: {'min_samples_leaf': 30, 'min_samples_split': 100},
  mean: 0.45714, std: 0.00327, params: {'min_samples_leaf': 30, 'min_samples_split': 120},
  

对最大特征数max_features做调参

In [0]:
param_test4= {'max_features':range(3,11,2)}  
gsearch4= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 50,max_depth=13, min_samples_split=80,  
                                 min_samples_leaf=10 ,oob_score=True, random_state=10),  
   param_grid = param_test4,scoring='accuracy',iid=False, cv=5)  
gsearch4.fit(X_train,y_train)  
gsearch4.grid_scores_,gsearch4.best_params_, gsearch4.best_score_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.41402, std: 0.00205, params: {'max_features': 3},
  mean: 0.46188, std: 0.00272, params: {'max_features': 5},
  mean: 0.47786, std: 0.00292, params: {'max_features': 7},
  mean: 0.48829, std: 0.00294, params: {'max_features': 9}],
 {'max_features': 9},
 0.48829055662781673)

最终的模型拟合： 

In [0]:
rf2= RandomForestClassifier(n_estimators= 50, max_depth=13, min_samples_split=80,  
                                 min_samples_leaf=10,max_features=9 ,oob_score=True, random_state=10)  
rf2.fit(X_train,y_train)  
print(rf2.oob_score_) 
y_predprob = rf2.predict_proba(X_test)

#print(np.shape(y_test))
#print(np.shape(y_predprob))

score = cross_val_score(rf2, train_X, train_y, cv=10, scoring='accuracy').mean()
print('Validation score is', score)

0.4822241096887703


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 

Validation score is 0.4966061432305625
